In [1]:
import boto3
import httpx
import json
import os

### Set variables and helper functions

In [2]:
def checkFilePath(file_path):
    result = s3.list_objects(Bucket=bucket, Prefix=file_path)
    exists = True if 'Contents' in result else False
    if exists:
        print('PATH EXISTS')
        return result['Contents']
    return exists

In [3]:
user = os.getenv('CHE_WORKSPACE_NAMESPACE')
titiler_url = "https://titiler.maap-project.org"  # MAAP titiler endpoint
titiler_tilejson_url = f"{titiler_url}/cog/tilejson.json"
bucket = "maap-ops-workspace"
band_min = 0
band_max = 50
color_map = "gist_earth_r"

#### Option to query possible projections supported by titiler service

In [25]:
tileMatrixSets = httpx.get(f"{titiler_url}/tileMatrixSets").json()
print("Supported TMS:")
for tms in tileMatrixSets["tileMatrixSets"]:
    print("-", tms["id"])

Supported TMS:
- CanadianNAD83_LCC
- EuropeanETRS89_LAEAQuad
- LINZAntarticaMapTilegrid
- NZTM2000
- NZTM2000Quad
- UPSAntarcticWGS84Quad
- UPSArcticWGS84Quad
- UTM31WGS84Quad
- WGS1984Quad
- WebMercatorQuad
- WorldCRS84Quad
- WorldMercatorWGS84Quad


### Right-click on the file and select option to Copy Path

In [4]:
path = input("Path to raster in bucket:")

Path to raster in bucket: shared-buckets/alexdevseed/landsat8/viz/Copernicus_30439_covars_cog_topo_stack.tif


In [5]:
s3 = boto3.client('s3')
file_name = path.split('/', 1)[-1]
if 'shared-buckets' in path:
    file_path = f'shared/{file_name}'
if 'my-private-bucket' in path:
    file_path = f'{user}/{file_name}'
if 'my-public-bucket' in path:
    file_path = f'shared/{user}/{file_name}'
print(checkFilePath(file_path))

PATH EXISTS
[{'Key': 'shared/alexdevseed/landsat8/viz/Copernicus_30439_covars_cog_topo_stack.tif', 'LastModified': datetime.datetime(2021, 7, 22, 22, 39, 57, tzinfo=tzutc()), 'ETag': '"0dbc859db2e921cda2b2ef403fa41f97-3"', 'Size': 20084240, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'MSFC-IMPACT-MAAP-Ops-root', 'ID': '801c37e81ec7d7b327915c96502ec5f346f48f2cdc819da9284110dbc39b64e7'}}, {'Key': 'shared/alexdevseed/landsat8/viz/Copernicus_30439_covars_cog_topo_stack.tif.aux.xml', 'LastModified': datetime.datetime(2023, 3, 4, 1, 31, 21, tzinfo=tzutc()), 'ETag': '"8ca416537cb9d82de62177a3a65d3751"', 'Size': 2052, 'StorageClass': 'STANDARD', 'Owner': {'DisplayName': 'MSFC-IMPACT-MAAP-Ops-root', 'ID': '801c37e81ec7d7b327915c96502ec5f346f48f2cdc819da9284110dbc39b64e7'}}]


In [6]:
url = f"s3://maap-ops-workspace/{file_path}"

#### If Path exists, continue...

### Open raster from url

In [11]:
import rioxarray as rxr

raster = rxr.open_rasterio(url, masked=True)
raster

#### Project to default map projection

In [13]:
crs = raster.rio.crs
print("The CRS of this dataset is:", crs)
crs_number = crs.to_epsg()
if crs_number != 3857:
    raster = raster.rio.reproject("EPSG:3857")
    crs = raster.rio.crs
    print("\n", "The NEW CRS of this dataset is:", crs)

The CRS of this dataset is: EPSG:3857


#### Get raster info (bounds, zoom, data type)

In [15]:
r = httpx.get(
    f"{titiler_url}/cog/info",
    params = {
        "url": url,
    }
).json()

# print(json.dumps(r, indent=4))

bounds = r.get("bounds")
minzoom = r.get("minzoom")
zoom = minzoom + 1 if minzoom == 0 else minzoom
bands = r.get("band_metadata")

print("Bounds:", bounds)
print("Zoom:", zoom)
print("Data type:", r.get("dtype"))
print("Bands:", bands)

Bounds: [-163.51338693693168, 67.17121197506852, -162.6566451826878, 67.49580310072406]
Zoom: 8
Data type: float32
Bands: [['b1', {}], ['b2', {}], ['b3', {}], ['b4', {}], ['b5', {}]]


#### Calculate raster center for map placement

In [16]:
from shapely.geometry import box

polygon = box(*bounds)
center = (polygon.centroid.y, polygon.centroid.x)
print("Center:", center)

Center: (67.33350753789628, -163.0850160598097)


#### Get value statistics for rescaling

In [17]:
r = httpx.get(
    f"{titiler_url}/cog/statistics",
    params = {
        "url": url,
    }
).json()

# print(json.dumps(r, indent=4))
band = r.get("b1")
if band:
    band_min, band_max = band.get("min"), band.get("max")
    print("min:", band_min, "max:", band_max)

minv: -1.829763650894165 maxv: 557.8629150390625


### Display local raster

#### Create TileLayer

In [19]:
from ipyleaflet import TileLayer

params = {
    "url": url,
    "tile_scale": "1",
    "tile_format": "png",
    "TileMatrixSetId": "WebMercatorQuad",
    "return_mask": "true",
    "rescale": f"{band_min}, {band_max}",
    "resampling_method": "nearest",
    "pixel_selection": "first",
    "bidx": "1",
    "colormap_name": color_map
}
r = httpx.get(titiler_tilejson_url, params=params)

if r.status_code == 200:
    json = r.json()
    layer_url = json['tiles'][0]
    custom_layer = TileLayer(url=layer_url, show_loading=True, transparent=True)

In [20]:
import stac_ipyleaflet

m = stac_ipyleaflet.StacIpyleaflet(zoom=zoom, center=center)
m.add_layer(custom_layer)
m

Output()

StacIpyleaflet(center=[67.33350753789628, -163.0850160598097], controls=(ZoomControl(options=['position', 'zoo…

#### If adding layer after map was created, use the fit_bounds method to navigate to it
`m.fit_bounds(bounds)`

#### Option to remove custom layer
`m.remove_layer(custom_layer)`